In [34]:
# Use PyTorch
#import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 2)
        self.conv2 = nn.Conv2d(16, 16, 3, 2)
        self.fc1 = nn.Linear(576, 100)
        self.fc2 = nn.Linear(100, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 576)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
        

In [35]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        total_loss.append(loss.item())
        #sys.stdout.write('.')
    #print('')
    print ('Epoch {}, average loss = {}'.format(epoch, sum(total_loss)/len(total_loss)))

In [37]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()
    test_loss /= len(test_loader.dataset)
    correct = 1. * correct / len(test_loader.dataset)
    print ('Test loss = {}, accuracy = {}%'.format(test_loss, correct * 100.))

In [5]:
def main():
    device = torch.device('cpu')
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True)
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.5)
    
    args = {} # Extra args
    for epoch in range(10):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

In [38]:
if __name__ == '__main__':
    main()

Epoch 0, average loss = 0.58500739173499
Test loss = 0.22590290474891664, accuracy = 93.26%
Epoch 1, average loss = 0.18579699158239593
Test loss = 0.14667325031757356, accuracy = 95.50999999999999%
Epoch 2, average loss = 0.13033150587636017
Test loss = 0.11550254786014556, accuracy = 96.34%
Epoch 3, average loss = 0.10385156767184674
Test loss = 0.08536583425998688, accuracy = 97.38%
Epoch 4, average loss = 0.08692150010165375
Test loss = 0.07973622317314148, accuracy = 97.32%
Epoch 5, average loss = 0.07473142532460979
Test loss = 0.08099015662670135, accuracy = 97.37%
Epoch 6, average loss = 0.06534387174842836
Test loss = 0.06504862253665925, accuracy = 97.86%
Epoch 7, average loss = 0.05753166550226303
Test loss = 0.06782806587219238, accuracy = 97.77%
Epoch 8, average loss = 0.052617895109122244
Test loss = 0.0648585063457489, accuracy = 97.84%
Epoch 9, average loss = 0.0472996067434454
Test loss = 0.05612728929519653, accuracy = 98.21%
